In [2]:
!pip install colab-repo

ERROR: Could not find a version that satisfies the requirement colab-repo (from versions: none)
ERROR: No matching distribution found for colab-repo


In [ ]:
import torch
import re
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

In [ ]:
# loading BioBERT for NER

model_name = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
ner_model = AutoModelForTokenClassification.from_pretrained(model_name)

# force gpu usage

device = "cuda" if torch.cuda.is_available() else "cpu"
ner_model.to(device)

nlp_ner = pipeline("ner", model=ner_model, tokenizer=tokenizer, device=0 if device == "cuda" else -1)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Training.csv to Training.csv


In [ ]:
df = pd.read_csv("Training.csv")

In [ ]:
# extracting symptom column names, exclude prognosis column

symptom_columns = list(df.columns[:-1])

In [ ]:
# preprocessing, converting text to lowercase and removing punctuation

def preprocess_text(text):
    text = text.lower().strip()  # Convert to lowercase and remove extra spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove punctuation
    tokenized_text = tokenizer.tokenize(text)
    truncated_text = tokenizer.convert_tokens_to_string(tokenized_text[:512])  # Truncate to 512 tokens
    return truncated_text

In [ ]:
# symptom extraction using BioBERT NER
def extract_symptoms_biobert(user_input):
    user_input = preprocess_text(user_input)
    ner_results = nlp_ner(user_input)
    detected_symptoms = set()

    for entity in ner_results:
        entity_text = entity["word"].replace("##", "").lower()
        detected_symptoms.add(entity_text)

    # Match extracted symptoms with dataset columns
    matched_symptoms = {symptom for symptom in symptom_columns if any(word in detected_symptoms for word in symptom.replace("_", " ").split())}

    return matched_symptoms

In [ ]:
# generating multi-hot vector

def symptoms_to_vector(detected_symptoms):
    return np.array([1 if symptom in detected_symptoms else 0 for symptom in symptom_columns], dtype=int)

In [ ]:
# Example usage
user_text = "feeling weak, my throat hurts, and i have chills"
detected = extract_symptoms_biobert(user_text)
vector = symptoms_to_vector(detected)

print("Detected Symptoms:", detected)
print("Multi-hot Vector:", vector)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Detected Symptoms: {'blurred_and_distorted_vision', 'patches_in_throat', 'drying_and_tingling_lips', 'throat_irritation', 'puffy_face_and_eyes', 'cold_hands_and_feets'}
Multi-hot Vector: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
